# Tutorial 3
## Testing autoencoder reconstruction loss

In this supplementary tutorial, test the autoencoder reconstruction loss for a given sample.

In [1]:
# Import system modules
import os
import gc
import sys
import glob
import importlib
from google.colab import drive

# Import computational modules
import torch
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

# Import plotting modules
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import art3d
from matplotlib.patches import Ellipse

In [2]:
# Mount repository to Google Colab
colab = True
if colab == True:
  drive.mount('/content/MyDrive', force_remount=True)
  os.chdir('/content/MyDrive/MyDrive/Courses/NeuralNetworks/e4040-fall2025-project-lsls/')

Mounted at /content/MyDrive


## Load the authors' per-class autoencoder models with saved weights

In this section, we run the authors' per-class autoencoder models on four test samples, one belonging to each sleep state/class. We do a preliminary assessment of autoencoder accuracy to determine whether to follow the suggested model pipeline, where they intended the autoencoder to succeed the CNN-based classifcation in the case of 'unknown' samples that fell below the 90% confidence bound.

In [ ]:
# Add file paths for author autoencoders

f_list = ["author_autoencoders/autoencoder_simpl_state_awake_epochs_150.h5",
          "author_autoencoders/autoencoder_simpl_state_slow_updown_epochs_150.h5",
          "author_autoencoders/autoencoder_simpl_state_slow_MA_epochs_150.h5",
          "author_autoencoders/autoencoder_simpl_state_asynch_MA_epochs_150.h5"]

In [ ]:
# Load model
# Ignore warnings, we are only using model.predict and we don't need the optimizer state

autoencoders = {
    "Awake": load_model(f_list[0]),
    "Slow Up-Down": load_model(f_list[1]),
    "Slow MA": load_model(f_list[2]),
    "Asynch MA": load_model(f_list[3]),
}
states  = list(autoencoders.keys())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Load data and metadata

from utils import preprocess
data, metadata = preprocess.load_data()
data = preprocess.normalize_data(data)

In [ ]:
# Take one sample from each class for testing AEs
one_index_per_state = {
    state: int(metadata.index[metadata["state"] == state].to_series().sample(1).iloc[0])
    for state in metadata["state"].unique()
}
sample_indices = list(one_index_per_state.values())
samples = data[sample_indices]

# Show sample metadata
metadata.iloc[list(one_index_per_state.values())]

,state,port,channel,desired_frequency,real_fs,win_size,subject,session,filename,comments
492,awake,10,PtA_R,1000,1010.097917,2,Phoebe,210626,210626_Rec01_Anesthesialvls_DEEP_Wakefulness_0...,NaN
15565,slow_updown,6,S1_R6,1000,1010.100500,2,Phoebe,210626,210626_Rec06_Anesthesialvls_DEEP_01MA_Phoebe.smr,NaN
28312,asynch_MA,2,OFC_R2,1000,1010.100306,2,Phoebe,210626,210626_Rec10_Anesthesialvls_DEEP_01MA_Phoebe.smr,good detection
31720,slow_MA,13,V1_R,1000,1010.100306,2,Phoebe,210626,210626_Rec10_Anesthesialvls_DEEP_01MA_Phoebe.smr,good detection


In [ ]:
from utils import autoencoder_utils
autoencoder_utils.reconstruction_loss(samples, states, autoencoders)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step

Reconstruction Loss for Awake Test Sample
Awake                     : 0.676483
Slow Up-Down              : 0.363552
Slow MA                   : 0.444668
Asynch MA                 : 0.523221
Predicted Classification: Slow Up-Down


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step

Reconstruction Loss for Slow Up-Down Test Sample
Awake                     : 0.311289
Slow Up-Down              : 0.066303
Slow MA                   : 0.237852
Asynch MA                 : 0.240252
Predicted Classification: Slow Up-Down


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step

Reconstruction Loss for Slow MA Test Sample
Awake                     